In [ ]:
from elasticsearch import Elasticsearch, helpers
import os
import pandas as pd
import json
import gradio as gr
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [ ]:
client = Elasticsearch(
    os.getenv('ELASTIC_HOST'),
    ca_certs=os.getenv('CERT_PATH'),
    basic_auth=(os.getenv('ELASTIC_USER'), os.getenv('ELASTIC_PASS'))
)
client.ping()

In [ ]:
# df = pd.read_csv("./ziymo.csv")
# df.head()
# df.columns
# df.shape
# df.isna().sum()
#df.dropna() #will drop all rows with a na val in any of the columns

In [ ]:
# df_dict = df.to_dict('records')
# print(df_dict[0])

In [ ]:
#client.indices.create(index="zimyo", settings=Settings, ignore=[400, 401])

In [ ]:
def search(text, num):
  body1 = {
    "query":{
      "match_phrase_prefix": {
        "command name":text
      }
    },
    "size":num
  }

  body2 = {
    "query":{
      "bool": {
        "should": [
          {
            "match_phrase_prefix": {
              "command name": text
            }
          },
          {
            "match":{
              "command name": text
            }
          },
          {
            "match": {
              "keywords": text
            }
          }
        ]
      }
    },
    "size":num
  }

  resp = client.search(index="ziymo", body=body2)
  return resp

def get_num_results(text, num):
  resp = search(text, num)
  #results = [(val["_source"]["command name"], val["_source"]["url"]) for val in resp["hits"]["hits"]]
  results = [val["_source"]["command name"] for val in resp["hits"]["hits"]]
  return "\n".join(results)

In [ ]:
# resp = search("employee", 10)
# print(resp)
# len(resp["hits"]["hits"])

In [ ]:
import gradio as gr

gr.close_all()
inputs = [gr.Textbox(label="search"), gr.Textbox(label="limit")]
demo = gr.Interface(fn=get_num_results, inputs=inputs, outputs=gr.Textbox(label="search results"), allow_flagging="never")
demo.launch()